<a href="https://colab.research.google.com/github/jy311dou311/visang/blob/main/atc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 설치 (최초 1회)
!pip install pdfplumber openpyxl
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.1 MB/s eta 0:00:00


In [ ]:
# 라이브러리 불러오기
import pdfplumber
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pdfminer.high_level import extract_text

In [ ]:
# 특수 문자를 일반 숫자/기호로 변환하는 함수
def normalize_custom_chars(text):
    table = {
        '': '1', '': '2', '': '3', '': '4',
        '': '5', '': '6', '': '7', '': '8',
        '': '9', '': '0', '': '=', '': ',',
        '': '', '': '.', '': '-', '': '+'
    }
    for key, val in table.items():
        text = text.replace(key, val)
    return text


In [ ]:
def clean_text(text):
    text = normalize_custom_chars(text)  # 먼저 특수문자 변환
    text = re.sub(r"[^가-힣a-zA-Z0-9\s\+\-\*/=()%]", " ", text)  # 의미 있는 기호는 남김
    text = re.sub(r"\s+", " ", text)
    return text.strip()


In [ ]:
# PDF에서 텍스트 추출
def extract_text(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + "\n"
    return text


In [ ]:
# problem_bank 분리
def split_problem_bank(text):
    problems = []
    entries = re.split(r"\n\s*\[문제번호:\s*(\w+)\]", text)
    for i in range(1, len(entries), 2):
        code = entries[i].strip()
        content = clean_text(entries[i + 1])
        problems.append({'유형코드': code, '내용': content})
    return pd.DataFrame(problems)

In [ ]:
# new_problems 분리
def split_new_problems(text):
    problems = re.split(r"\n\s*\d+\s*번 문제", text)
    problems = [clean_text(p) for p in problems if p.strip()]
    return pd.DataFrame({'문제': problems})

In [ ]:
# TF-IDF 유사도 기반 문제 매칭
def match_problems(df_bank, df_new):
    df_bank['문제내용'] = df_bank['내용'].apply(clean_text)
    df_new['문제'] = df_new['문제'].apply(clean_text)
    vectorizer = TfidfVectorizer()
    bank_vectors = vectorizer.fit_transform(df_bank['내용'])


    matched = []
    for i, row in df_new.iterrows():
        problem_vec = vectorizer.transform([row['문제']])
        similarities = cosine_similarity(problem_vec, bank_vectors).flatten()
        best_idx = similarities.argmax()
        best_score = similarities[best_idx]
        matched.append({
            '문제번호': f'{i+1}번 문제',
            '문제내용': row['문제'],
            '가장 유사한 유형코드': df_bank.iloc[best_idx]['유형코드'],
            '유사도 점수': round(best_score, 3)
        })

    return pd.DataFrame(matched)


In [ ]:
def save_feedback_by_number(df_result, 문제번호, 수정유형코드, filepath='feedback.csv'):
    try:
        row = df_result[df_result['문제번호'] == 문제번호].iloc[0]
        문제내용 = row['문제내용']
        기존코드 = row['가장 유사한 유형코드']

        # 기존 피드백 로드
        feedback_df = load_feedback(filepath)
        feedback_df = feedback_df[feedback_df['문제내용'] != 문제내용]

        new_row = pd.DataFrame([{
            '문제내용': 문제내용,
            '기존유형코드': 기존코드,
            '수정유형코드': 수정유형코드
        }])
        feedback_df = pd.concat([feedback_df, new_row], ignore_index=True)
        feedback_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 피드백 저장 완료: {문제번호} → {수정유형코드}")
    except IndexError:
        print(f"❌ 해당 문제번호 '{문제번호}'를 찾을 수 없습니다.")


In [ ]:
def apply_feedback_to_bank(df_bank, feedback_df):
    """
    피드백 데이터를 기반으로 문제은행을 업데이트 (덮어쓰기 or 추가).
    """
    # '문제내용'을 기준으로
    df_bank = df_bank.copy()
    df_bank['내용'] = df_bank['내용'].str.strip()
    feedback_df['문제내용'] = feedback_df['문제내용'].str.strip()

    # 중복되는 문제는 제거하고 피드백 반영된 데이터로 대체
    updated_bank = df_bank[~df_bank['내용'].isin(feedback_df['문제내용'])]

    # 피드백 데이터프레임 형식을 맞춰서 병합
    feedback_as_bank = feedback_df.rename(columns={
        '문제내용': '내용',
        '수정유형코드': '유형코드'
    })[['내용', '유형코드']]

    final_bank = pd.concat([updated_bank, feedback_as_bank], ignore_index=True)
    return final_bank


In [ ]:
def load_feedback(filepath='feedback.csv'):
    import pandas as pd
    try:
        return pd.read_csv(filepath)
    except FileNotFoundError:
        return pd.DataFrame(columns=['문제내용', '기존유형코드', '수정유형코드'])


In [ ]:
# 전체 실행
problem_bank_text = extract_text("problem_bank.pdf")
new_problems_text = extract_text("new_problems.pdf")

df_bank = split_problem_bank(problem_bank_text)
df_new = split_new_problems(new_problems_text)

df_result = match_problems(df_bank, df_new)
df_result.to_csv("matching_result.csv", index=False, encoding="utf-8-sig")
df_result


,문제번호,문제내용,가장 유사한 유형코드,유사도 점수
0,1번 문제,1번 문제 다음 조건을 만족시키는 두 자리 자연수의 개수를 구하시오 일의 자리의 수...,1111a,0.565
1,2번 문제,할머니 아버지 어머니 아들 딸로 구성된 5명의 가족이 있다 이 가족이 번호가 적힌 ...,1112d,0.466
2,3번 문제,의 값을 구하시오 5 2 5 3 (답)30 5 (해설) 5 2 5 3 =5 4 3 ...,1113a,0.811
3,4번 문제,1부터 8까지의 자연수가 각각 하나씩 적혀 있는 8장의 카드 중에서 동시에 5장의 ...,1114a,0.271
4,5번 문제,체력단련장에서 사용하는 운동기구에는 운동 관련 정보 안내 화면이 3개 있다 한 화면...,1114a,0.216


In [ ]:
#사용자 피드백 수동 반영
save_feedback_by_number(df_result, "4번 문제", "1114a")
save_feedback_by_number(df_result, "5번 문제", "1115a")
save_feedback_by_number(df_result, "6번 문제", "2115a")

✅ 피드백 저장 완료: 4번 문제 → 1114a
✅ 피드백 저장 완료: 5번 문제 → 1115a
❌ 해당 문제번호 '6번 문제'를 찾을 수 없습니다.


In [ ]:
def match_problems_with_feedback(df_bank, df_new, feedback_df, threshold=0.95):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    vectorizer = TfidfVectorizer()
    bank_vectors = vectorizer.fit_transform(df_bank['내용'])
    feedback_vectors = vectorizer.transform(feedback_df['문제내용'])

    matched = []
    for i, row in df_new.iterrows():
        problem_text = row['문제']
        problem_vec = vectorizer.transform([problem_text])

        # 피드백 유사도 계산
        feedback_sim = cosine_similarity(problem_vec, feedback_vectors).flatten()
        max_sim = feedback_sim.max()
        if max_sim >= threshold:
            best_feedback_idx = feedback_sim.argmax()
            corrected_code = feedback_df.iloc[best_feedback_idx]['수정유형코드']
            matched.append({
                '문제번호': f"{i+1}번 문제",
                '문제내용': problem_text,
                '매칭된 유형코드': corrected_code,
                '유사도 점수': f"피드백({max_sim:.3f})"
            })
            continue

        # 일반 문제은행 유사도 계산
        similarities = cosine_similarity(problem_vec, bank_vectors).flatten()
        best_idx = similarities.argmax()
        best_score = similarities[best_idx]
        best_code = df_bank.iloc[best_idx]['유형코드']

        matched.append({
            '문제번호': f"{i+1}번 문제",
            '문제내용': problem_text,
            '매칭된 유형코드': best_code,
            '유사도 점수': round(best_score, 3)
        })

    return pd.DataFrame(matched)
